## Context

we have single datasets for each type of "factor" that people might take in consideration when deciding what country to live in, but we need to merge all of them into one single dataset in order to build models with this data

### Exploration

we have 5 datasets in total:
- cost of living
- healthcare index
- safety index
- internet speed
- average temperatures

In [9]:
import pandas as pd

cost_of_living = pd.read_csv("../raw_data/Alternative_sources_country_level/cost_expense.csv")
#healtcare = pd.read_csv("../raw_data/Alternative_sources_country_level/????????")
climate = pd.read_csv("../raw_data/Alternative_sources_country_level/climate_avarage_temperature.csv")
internet = pd.read_csv("../raw_data/Alternative_sources_country_level/internet_speed_rankings.csv")
safety = pd.read_csv("../raw_data/Alternative_sources_country_level/safety_index_data.csv")

In [10]:
#select only the columns we need for now, rename them with explanative names and set the index to country for all datasets

#cost of living
cost_of_living = cost_of_living[['country', 'average_monthly_cost($)']]
cost_of_living.columns = ['Country', 'average_monthly_cost_$']
cost_of_living.set_index('Country', inplace=True)

# healthcare dataset
#healthcare = healthcare[['Country', 'Healthcare Index']]
#healthcare.columns = ['Country', 'Healthcare Index']
#healthcare.set_index('Country', inplace=True)

# climate dataset
climate = climate[['Country', 'Temperature']]
climate.columns = ['Country', 'average_yearly_temperature']
climate.set_index('Country', inplace=True)

# internet dataset
internet = internet[['Country', 'Internet Speed (Mbps)']]
internet.columns = ['Country', 'internet_speed_mbps']
internet.set_index('Country', inplace=True)

# safety dataset
safety = safety[['Country', 'Safety Index']]
safety.columns = ['Country', 'safety_index']
safety.set_index('Country', inplace=True)
